# SentEval usage example

* Clone repo from FAIR github
```
    git clone https://github.com/facebookresearch/SentEval.git
    cd SentEval/
```
* Dependencies:
    * Python 2/3 with NumPy/SciPy
    * Pytorch
    * scikit-learn>=0.18.0

* Install senteval
```
    python setup.py install
```
* Download datasets (it takes some time...)
    * these are downstream tasks
    * new Senteval also has probing tasks (https://github.com/facebookresearch/SentEval/tree/master/data/probing) for evaluating linguistic properties of your embeddings. 
```
    cd data/downstream/
    ./get_transfer_data.bash
```
* Download pretained Glove embeddings:

```
    mkdir pretrained
    cd pretrained
    wget http://nlp.stanford.edu/data/glove.840B.300d.zip
   
```

* The following code evaluates Glove pretrained embeddings on different NLP downstream tasks.

In [1]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
import examples.data as data

# Set PATHs
# path to senteval
PATH_TO_SENTEVAL = '.'
# path to the NLP datasets 
PATH_TO_DATA = 'data/'
# path to glove embeddings
PATH_TO_VEC = 'pretrained/glove.840B.300d.txt'


# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval


def prepare(params, samples):
    """
    In this example we are going to load Glove, 
    here you will initialize your model.
    remember to add what you model needs into the params dictionary
    """
    _, params.word2id = data.create_dictionary(samples)
    # load glove/word2vec format 
    params.word_vec = data.get_wordvec(PATH_TO_VEC, params.word2id)
    # dimensionality of glove embeddings
    params.wvec_dim = 300
    return

def batcher(params, batch):
    """
    In this example we use the average of word embeddings as a sentence representation.
    Each batch consists of one vector for sentence.
    Here you can process each sentence of the batch, 
    or a complete batch (you may need masking for that).
    
    """
    
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []

    for sent in batch:
        sentvec = []
        # the format of a sentence is a lists of words (tokenized and lowercased)
        for word in sent:
            if word in params.word_vec:
                # [number of words, embedding dimensionality]
                sentvec.append(params.word_vec[word])
        if not sentvec:
            vec = np.zeros(params.wvec_dim)
            # [number of words, embedding dimensionality]
            sentvec.append(vec)
        # average of word embeddings for sentence representation
        # [embedding dimansionality]
        sentvec = np.mean(sentvec, 0)
        embeddings.append(sentvec)
    # [batch size, embedding dimensionality]
    embeddings = np.vstack(embeddings)
    return embeddings


# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10}
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks = ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                           'MRPC', 'SICKEntailment', 'STS14']
    #transfer_tasks = ['MR']
    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    print(results)

2018-05-24 14:39:53,079 : ***** Transfer task : MR *****


2018-05-24 14:40:10,840 : Found 18490 words with word vectors, out of         20328 words
2018-05-24 14:40:10,840 : Generating sentence embeddings
2018-05-24 14:40:11,465 : Generated sentence embeddings
2018-05-24 14:40:11,465 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-24 14:40:54,002 : Best param found at split 1: l2reg = 1                 with score 78.15
2018-05-24 14:41:36,255 : Best param found at split 2: l2reg = 2                 with score 78.04
2018-05-24 14:42:18,218 : Best param found at split 3: l2reg = 1                 with score 78.01
2018-05-24 14:43:00,242 : Best param found at split 4: l2reg = 1                 with score 77.96
2018-05-24 14:43:41,756 : Best param found at split 5: l2reg = 2                 with score 78.18
2018-05-24 14:44:24,235 : Best param found at split 6: l2reg = 4                 with score 77.9
2018-05-24 14:45:06,578 : Best param found at split 7: l2reg = 

2018-05-24 15:14:13,567 : Computing embedding for train
2018-05-24 15:14:13,942 : Computed train embeddings
2018-05-24 15:14:13,942 : Computing embedding for test
2018-05-24 15:14:14,286 : Computed test embeddings
2018-05-24 15:14:14,286 : Computing embedding for dev
2018-05-24 15:14:14,348 : Computed dev embeddings
2018-05-24 15:14:14,411 : Training sklearn-LogReg with standard validation..
2018-05-24 15:14:23,596 : [('reg:0.25', 77.8), ('reg:0.5', 80.0), ('reg:1', 80.8), ('reg:2', 81.0), ('reg:4', 80.0), ('reg:8', 80.0)]
2018-05-24 15:14:23,596 : Validation : best param found is reg = 2 with score             81.0
2018-05-24 15:14:23,596 : Evaluating...
2018-05-24 15:14:25,221 : 
Dev acc : 81.0 Test acc : 79.01 for                        SICK entailment

2018-05-24 15:14:25,236 : ***** Transfer task : STS14 *****


2018-05-24 15:14:40,170 : Found 8608 words with word vectors, out of         9119 words
2018-05-24 15:14:40,264 : deft-forum : pearson = 0.3002, spearman = 0.3472
2018-05-

{'STS14': {'deft-news': {'pearson': (0.6494706603328699, 2.5016250621003608e-37), 'spearman': SpearmanrResult(correlation=0.6455872244607762, pvalue=9.14578325274358e-37), 'nsamples': 300}, 'OnWN': {'pearson': (0.5770942998658635, 8.24696998029528e-68), 'spearman': SpearmanrResult(correlation=0.6434659871555309, pvalue=6.851316923329252e-89), 'nsamples': 750}, 'deft-forum': {'pearson': (0.3001569947762378, 8.028866381393617e-11), 'spearman': SpearmanrResult(correlation=0.34721634810882696, pvalue=3.3959673218845644e-14), 'nsamples': 450}, 'headlines': {'pearson': (0.5867155417421495, 1.442541063841459e-70), 'spearman': SpearmanrResult(correlation=0.5510007623358119, pvalue=8.93555685715001e-61), 'nsamples': 750}, 'all': {'pearson': {'mean': 0.5459825628410645, 'wmean': 0.5532300365872339}, 'spearman': {'mean': 0.5563585973068762, 'wmean': 0.562382541984252}}, 'images': {'pearson': (0.6237902519493352, 4.1488863571600114e-82), 'spearman': SpearmanrResult(correlation=0.6126084329972454, 